In [53]:
import pandas as pd
import os
import numpy as np
from haversine import haversine
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
os.chdir('C:/Users/user/Desktop/서울시청/데이터')

* 최소 거리 계산

In [45]:
def dis_cal(grid,b):
    dis_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        dis_min=999999999999999999
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<dis_min:
                dis_min=dis
        dis_l.append(dis_min)
        if i%40000==0:
            print(i)
    return dis_l

* 반경 100m count

In [27]:
def count_100cal(grid,b):
    count_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        count=0
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<=100:
                count+=1
        count_l.append(count)
        if i%40000==0:
            print(i)
    return count_l

* 반경 100m 존재 여부

In [28]:
def is_100cal(grid,b):
    count_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        count=0
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<=100:
                count=1
                count_l.append(count)
                break
        if count==0:
            count_l.append(count)
        if i%40000==0:
            print(i)
    return count_l

* 최소거리 내의 value

In [26]:
def dis_value(grid,b):
    dis_idx=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        dis_min=999999999999999999
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<dis_min:
                dis_min=dis
                idx=j
        dis_idx.append(idx)
        if i%40000==0:
            print(i)
    return dis_idx

* 반경 100m 내의 value 합

In [92]:
def value_100cal(grid,b):
    value_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    dat=list(b['합계'])
    for i in range(length_A):
        value=0
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<=100:
                value+=dat[j]
        value_l.append(value)
        if i%40000==0:
            print(i)
    return value_l

### 중,고등학교 전처리

In [29]:
school=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/학교/중고등학교위치_수정.csv',encoding='cp949',index_col=0)
school.columns

Index(['학교ID', '학교명', '학교급구분', '설립일자', '설립형태', '본교분교구분', '운영상태', '소재지지번주소',
       '소재지도로명주소', '시도교육청코드', '시도교육청명', '교육지원청코드', '교육지원청명', '생성일자', '변경일자',
       '위도', '경도', '데이터기준일자'],
      dtype='object')

In [30]:
school=school[['학교명','위도','경도']]

In [31]:
school.reset_index(drop=True,inplace=True)

In [32]:
school.head()

,학교명,위도,경도
0,서울문영여자고등학교,37.472905,126.950918
1,봉은중학교,37.518977,127.062389
2,선유중학교,37.533286,126.894329
3,독산고등학교,37.476219,126.910973
4,대영고등학교,37.503661,126.905866


### 문화, 편의시설 데이터 불러오기(전처리 및 병합)

In [46]:
sports=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/스포츠센터병합.csv',encoding='cp949',index_col=0)
adult=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/유흥업소.csv',encoding='cp949',index_col=0)
study=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/학업시설.csv',encoding='cp949',index_col=0)
market=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/편의점,마켓.csv',encoding='cp949',index_col=0)
game=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/오락시설.csv',encoding='cp949',index_col=0)
house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/숙박 시설.csv',encoding='cp949',index_col=0)
culture=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/문화,쇼핑.csv',encoding='cp949',index_col=0)
hell=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/지옥.csv',encoding='cp949',index_col=0)
parking=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/주차장병합.csv',encoding='cp949',index_col=0)

### 체육시설 빅데이터 캠퍼스 데이터 병합
* QGIS 좌표계 위도, 경도(4326) 변환 후 기존 데이터와 병합

In [54]:
gong_sports=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/441. 서울시 공공체육시설 공간데이터/2. 파일데이터/2017/공공체육시설/TL_PUB_ALSFC/공공체육시설_4326.csv',encoding='cp949')
sports=sports[['상호명','위도','경도']]

In [55]:
gong_sports=gong_sports[['ALSFC_NM','y','x']]
gong_sports.columns=sports.columns

In [58]:
sports=pd.concat([sports,gong_sports])
sports.reset_index(drop=True,inplace=True)
sports.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/스포츠센터병합.csv',encoding='cp949')

## 자전거 편의시설 전처리
* QGIS 좌표계 위도, 경도(4326) 변환 후 처리

In [59]:
bike=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/408. 서울시 자전거 편의시설 공간데이터/2. 파일데이터/2017/자전거 인프라 현황/TL_BCYCL_INFR_ST_2017/자전거인프라_4326.csv',encoding='cp949')
bike=bike[(bike.TGRP_CODE==6)|(bike.TGRP_CODE==8)|(bike.TGRP_CODE==9)|(bike.TGRP_CODE==10)|(bike.TGRP_CODE==11)|(bike.TGRP_CODE==12)|(bike.TGRP_CODE==14)]
bike.reset_index(drop=True,inplace=True)
bike.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/자전거 편의시설.csv',encoding='cp949')

In [34]:
bike=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/자전거 편의시설.csv',encoding='cp949',index_col=0)

### 주차장 빅데이터 캠퍼스 데이터 병합
* QGIS 좌표계 위도, 경도(4326) 변환 후 기존 데이터와 병합

In [60]:
parking_outdoor=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/410. 서울시 노외주차장 공간데이터/2. 파일데이터/2017/주차장 정보(노외주차장)/TL_PRKPLCE_ST_2017/노외주차장.csv',encoding='cp949')

In [61]:
parking=parking[['상호명','위도','경도']]
parking_outdoor=parking_outdoor[['BD_15_NM','위도','경도']]
parking_outdoor.columns=parking.columns
parking=pd.concat([parking,parking_outdoor])
parking.reset_index(drop=True,inplace=True)
parking.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/주차장병합.csv',encoding='cp949')

### 자전거 인프라 전처리
* QGIS를 활용하여 인프라를 포함하는 격자 추출

In [62]:
bike.head()

,ID,TGRP_CODE,ADDR_KOR,위도,경도
0,174,12,서울특별시 강동구 천호동 484-9,37.543813,127.115989
1,175,12,서울특별시 강동구 천호동 485-12,37.543935,127.116303
2,176,12,서울특별시 마포구 마포동 425-1,37.536410,126.941293
3,177,12,서울특별시 광진구 광장동 557-1,37.543949,127.108888
4,178,12,서울특별시 용산구 이촌동 359,37.511094,126.981893


# 대여소 기준 독립변수 생성
* 자전거 인프라 : 최소 거리(m)
* 중,고등학교 : 최소 거리(m)
* 문화,편의시설
    * 스포츠 센터 : 최소 거리(m)
    * 유흥업소 : 최소 거리(m)
    * 학원 : 최소 거리(m)
    * 슈퍼마켓, 편의점 : 최소 거리(m)
    * 오락 : 최소 거리(m)
    * 숙박시설 : 최소 거리(m)
    * 문화 시설, 영화, 쇼핑 : 최소 거리(m)
    * 연구소, 고시원 : 최소 거리(m)
    * 주차장 : 최소 거리(m)

In [65]:
rental=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/4월 이전 대여소.csv',encoding='cp949',index_col=0)
rental.head()

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
1,2301,현대고등학교 건너편,강남구,서울특별시 강남구 압구정로 134,37.524071,127.021790,2017-06-13,10.0,NaN,LCD
2,2302,교보타워 버스정류장(신논현역 3번출구 후면),강남구,서울특별시 강남구 봉은사로 지하 102,37.505581,127.024277,2017-06-13,10.0,NaN,LCD
3,2303,논현역 7번출구,강남구,서울특별시 강남구 학동로 지하 102,37.511517,127.021477,2017-06-13,15.0,NaN,LCD
4,2304,신영 ROYAL PALACE 앞,강남구,서울특별시 강남구 언주로 626,37.512527,127.035835,2017-06-13,10.0,NaN,LCD
5,2305,MCM 본사 직영점 앞,강남구,서울특별시 강남구 언주로 734,37.520641,127.034508,2017-06-13,10.0,NaN,LCD


In [66]:
rental_place=pd.DataFrame(rental['대여소번호'])
rental_place

,대여소번호
1,2301
2,2302
3,2303
4,2304
5,2305
...,...
2033,1456
2034,1457
2035,1458
2036,1459


In [40]:
school_min=dis_cal(rental,school)
rental_place['school']=school_min

sports_count=dis_cal(rental,sports)
rental_place['sports']=sports_count

adult_count=dis_cal(rental,adult)
rental_place['adult']=adult_count

study_count=dis_cal(rental,study)
rental_place['study']=study_count

market_count=dis_cal(rental,market)
rental_place['market']=market_count

game_count=dis_cal(rental,game)
rental_place['game']=game_count

house_count=dis_cal(rental,house)
rental_place['house']=house_count

culture_count=dis_cal(rental,culture)
rental_place['culture']=culture_count

hell_count=dis_cal(rental,hell)
rental_place['hell']=hell_count

parking_count=dis_cal(rental,parking)
rental_place['parking']=parking_count

bike_min=dis_cal(rental,bike)
rental_place['bike']=bike_min


0
0
0
0
0
0
0
0
0
0
0


### 지하철 출구 독립변수 추가
* 지하철 출구 : 최소 거리(m)

In [57]:
subway=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/지하철 출구.csv',encoding='cp949')

In [33]:
subway.columns=['LBL_NAME','POI_NAME1','경도','위도']
subway

,LBL_NAME,POI_NAME1,경도,위도
0,3,사가정역 3번출구,127.090084,37.577807
1,4,사가정역 4번출구,127.090860,37.577723
2,8,오목교역 8번출구,126.875860,37.521749
3,3,용두역 3번출구,127.040777,37.571388
4,5,용두역 5번출구,127.039888,37.571194
...,...,...,...,...
1701,1,정릉역 1번출구,127.015583,37.600277
1702,2,정릉역 2번출구,127.015444,37.599972
1703,3,도봉산역 3번출구,127.048500,37.685667
1704,4-1,남부터미널역 4-1번출구,127.019638,37.480499


In [34]:
subway_min=dis_cal(rental,subway)
rental_place['subway']=subway_min

0


### 공원 독립변수 추가
* 공원 : 최소 거리(m)
    * QGIS 폴리곤 전처리
    * QGIS 폴리곤 최소거리
   

In [38]:
park=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공원/UPIS_SHP_ZON216(생활서비스시설-공원)/공원 최소거리.csv',encoding='cp949')

In [40]:
park['cat']=park['cat']+1

In [41]:
park

,cat,length
0,1,252.411658
1,2,317.618601
2,3,145.560776
3,4,324.674608
4,5,305.099076
...,...,...
1769,1872,146.570105
1770,1873,181.912726
1771,1874,370.992033
1772,1875,182.598290


In [54]:
park.columns=['cat','park']

In [44]:
rental_place.reset_index(drop=True,inplace=True)
rental_place['cat']=rental_place.index+1

In [57]:
rental_place=rental_place.merge(park,on='cat',how='outer')

In [58]:
rental_place.drop('cat',axis=1,inplace=True)

In [63]:
rental_place.loc[rental_place['park'].isna(),'park']=0

### 상업지역 면적 독립변수 추가
* 반경 100m 내의 상업지역 면적
    * QGIS 전처리
    * 반경 100m 버퍼 intersect 상업지역 폴리곤
    * 겹치는 폴리곤 dissolve
    * area

In [67]:
commercial=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/상업지역.csv',encoding='cp949')
commercial.columns=['대여소번호','상업지역']
commercial

,대여소번호,상업지역
0,2302,4691.636963
1,2303,3866.095459
2,2304,1988.298706
3,2305,4802.696167
4,2310,4176.172119
...,...,...
598,1448,24100.807007
599,1449,9265.285522
600,1455,12587.177246
601,1458,29576.531372


In [68]:
rental_place=rental_place.merge(commercial,on='대여소번호',how='outer')

In [69]:
rental_place.loc[rental_place['상업지역'].isna(),'상업지역']=0

### 주거지역 면적 독립변수 추가
* 반경 100m 내의 주거지역 면적
    * QGIS 전처리
    * 반경 100m 버퍼 intersect 주거지역 폴리곤
    * 겹치는 폴리곤 dissolve
    * area

In [70]:
first_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/전용주거지역.csv',encoding='cp949')

In [71]:
rental_place=rental_place.merge(first_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['전용주거'].isna(),'전용주거']=0

In [73]:
second_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/7층이하주거지역.csv',encoding='cp949')
second_house.head()

,대여소번호,7층이하주거지역
0,2301,16706.375122
1,2304,4165.816284
2,2305,1680.223022
3,2309,1250.753052
4,2311,2016.062744


In [74]:
rental_place=rental_place.merge(second_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['7층이하주거지역'].isna(),'7층이하주거지역']=0

In [75]:
third_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/7층이상주거지역.csv')
third_house.head()

,대여소번호,고층주거
0,2301,14677.222656
1,2302,26691.501099
2,2303,27516.596802
3,2304,22835.113037
4,2305,24900.542114


In [76]:
rental_place=rental_place.merge(third_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['고층주거'].isna(),'고층주거']=0

### 경사도 독립변수 추가
* 대여소 반경 100m 도로 기준
* sum((경사도*도로길이))/sum(도로길이)

In [77]:
slope=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/경사도/대여소별 경사도 가중치.csv',encoding='cp949',index_col=0)

In [78]:
slope=slope[['대여소번호','weighted_LEN_mean']]

In [79]:
slope

,대여소번호,weighted_LEN_mean
0,2301,0.845528
1,2302,2.966590
2,2303,3.572432
3,2304,4.828597
4,2305,3.447335
...,...,...
1871,1456,1.715694
1872,1457,1.477592
1873,1458,1.241584
1874,1459,1.241584


In [83]:
rental_place=rental_place.merge(slope,on='대여소번호',how='outer')

### 도로폭, 경사도 독립변수 추가
* 대여소 반경 100m 도로 기준
* sum((도로폭(경사도)*도로길이))/sum(도로길이)

In [4]:
width=pd.read_csv('도로폭/대여소별 도로폭 가중치.csv',encoding='cp949',index_col=0)

In [6]:
width=width[['대여소번호','mean_width']]

In [11]:
rental_place=rental_place.merge(width,on='대여소번호')

In [17]:
rental_place_p=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/최종대여소_50.csv')

In [18]:
rental_place=rental_place.merge(rental_place_p,left_on='대여소번호',right_on='ID')

## 지하철 승하차 독립변수 추가
* 가장 가까운 지하철 승하차 인원

In [80]:
week=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/버스승하차/평일.csv',encoding='cp949',index_col=0)
holi=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/버스승하차/주말.csv',encoding='cp949',index_col=0)

In [81]:
week

,역ID,위도,경도,합계
0,150,37.554648,126.972559,753428
1,151,37.564718,126.977108,266661
2,152,37.570161,126.982923,612057
3,153,37.571607,126.991806,520455
4,154,37.570926,127.001849,416413
...,...,...,...,...
322,4709,37.612343,127.008009,161025
323,4710,37.602798,127.013490,121675
324,4711,37.592624,127.016403,93095
325,4712,37.585286,127.019381,44529


In [64]:
week_idx=dis_value(rental,week)
holi_idx=dis_value(rental,holi)

0
0


In [87]:
week_value=[]
holi_value=[]
week_id=list(week['역ID'])
holi_id=list(holi['역ID'])
for i in range(len(week_idx)):
    week_value.append(list(week[week['역ID']==week_id[week_idx[i]]]['합계'])[0])
    holi_value.append(list(holi[holi['역ID']==holi_id[holi_idx[i]]]['합계'])[0])

In [89]:
rental_place['sub_week']=week_value
rental_place['sub_holid']=holi_value

## 버스 독립변수 생성
* 버스 정류장 최소 거리(m)
* 최근접 버스 정류장 승하차 인원

In [50]:
bus1=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/버스승하차/평일버스.csv',encoding='cp949')
bus2=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/버스승하차/주말버스.csv',encoding='cp949')

In [15]:
bus_1=bus1[bus1['역ID'].apply(lambda x : x in sub)].reset_index(drop=True)

In [8]:
sub=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/버스승하차/지하철역조인.csv',encoding='cp949')

In [9]:
sub=list(sub['역ID'])

In [16]:
bus1.sort_values(by='합계',ascending=False)

,역ID,경도,위도,합계
31,222,127.027926,37.497175,2570774
25,216,127.102234,37.513950,1939020
39,230,126.929715,37.484201,1876083
41,232,126.901401,37.485266,1759341
29,220,127.049008,37.504503,1509465
...,...,...,...,...
7305,8502041,126.883275,37.573857,1
7660,8502539,127.011453,37.519725,1
7306,8502042,126.880488,37.569582,1
5121,4001665,126.979733,37.505118,1


In [115]:
bus_1=bus1[bus1['역ID'].apply(lambda x : x not in sub)].reset_index(drop=True)

In [17]:
bus_1

,역ID,경도,위도,합계
0,150,126.972559,37.554648,753428
1,151,126.977108,37.564718,266661
2,152,126.982923,37.570161,612057
3,153,126.991806,37.571607,520455
4,154,127.001849,37.570926,416413
...,...,...,...,...
324,4709,127.008009,37.612343,161025
325,4710,127.013490,37.602798,121675
326,4711,127.016403,37.592624,93095
327,4712,127.019381,37.585286,44529


In [122]:
bus_2=bus2[bus2['역ID'].apply(lambda x : x not in sub)].reset_index(drop=True)

In [20]:
sub=pd.concat([bus_1,bus_2])

In [24]:
sub=sub.groupby(['역ID','경도','위도']).sum().reset_index()

In [124]:
holi_value=value_100cal(rental,bus_1)
rental_place['bus_week']=holi_value

holi_value=value_100cal(rental,bus_2)
rental_place['bus_holid']=holi_value


0
0


In [128]:
bus_all=pd.concat([bus_1,bus_2])

In [131]:
bus_all=bus_all.drop_duplicates('역ID')

In [132]:
bus_min=dis_cal(rental,bus_all)
rental_place['bus_dis']=bus_min


0


In [84]:
rental=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/병합용.csv',encoding='cp949')

In [85]:
rental

,대여소번호,subway,park,상업지역,weighted_LEN_mean,mean_width,주거,school,sports,adult,study,market,game,house,culture,hell,parking,bike
0,2301,711.946112,252.411658,0.000000,0.845528,25.347815,14677.222656,87.435847,153.147202,106.104049,56.544899,204.378320,264.584480,407.184550,451.375279,549.542108,245.960238,276.150048
1,2302,260.995145,317.618601,4691.636963,2.966590,40.044479,26691.501099,752.191921,268.035145,27.018746,34.234011,52.890109,27.018746,80.411068,70.729001,27.018746,8.272112,166.740527
2,2303,146.357936,145.560776,3866.095459,3.572432,39.651156,27516.596802,831.058272,179.164630,73.200264,7.562897,7.562897,7.562897,141.356740,92.319266,141.356740,268.101622,495.535749
3,2304,96.046760,324.674608,1988.298706,4.828597,34.868880,22835.113037,803.744535,280.317435,174.098002,127.601461,19.388554,207.075254,398.847263,297.243056,264.444721,402.037646,480.882159
4,2305,409.620345,305.099076,4802.696167,3.447335,36.600089,24900.542114,181.728030,302.311367,74.844749,144.220043,74.631317,144.220043,98.590793,235.221391,71.364635,123.679642,689.004072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,503.544596,146.570105,0.000000,1.715694,8.873014,21781.166748,353.886426,195.362847,11.992129,20.068117,11.992129,11.992129,142.753944,216.870280,11.992129,345.732668,750.976438
1872,1457,467.353819,181.912726,0.000000,1.477592,9.128512,29556.708984,515.272934,110.147483,41.028907,79.420018,61.476908,61.497648,81.975277,646.026714,221.583073,15.504156,1155.758488
1873,1458,39.430887,370.992033,29576.531372,1.241584,16.153486,1807.035522,533.787400,408.563075,29.649693,93.666293,93.666293,29.649693,87.815255,615.073449,159.966721,29.649693,333.873145
1874,1459,251.746998,182.598290,0.000000,1.241584,16.153486,21792.173340,449.461202,206.992487,85.553443,73.510332,83.299103,68.921890,222.186216,1145.074644,393.971232,444.854015,1223.915921


In [133]:
rental_place.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/버스포함.csv',encoding='cp949',index=False)

In [94]:
rental

,대여소번호,subway,park,상업지역,weighted_LEN_mean,mean_width,주거,school,sports,adult,...,market,game,house,culture,hell,parking,bike,bus_dis,bus_all,sub_all
0,2301,711.946112,252.411658,0.000000,0.845528,25.347815,14677.222656,87.435847,153.147202,106.104049,...,204.378320,264.584480,407.184550,451.375279,549.542108,245.960238,276.150048,84.969754,82647,1053987.0
1,2302,260.995145,317.618601,4691.636963,2.966590,40.044479,26691.501099,752.191921,268.035145,27.018746,...,52.890109,27.018746,80.411068,70.729001,27.018746,8.272112,166.740527,11.425873,285711,1105226.0
2,2303,146.357936,145.560776,3866.095459,3.572432,39.651156,27516.596802,831.058272,179.164630,73.200264,...,7.562897,7.562897,141.356740,92.319266,141.356740,268.101622,495.535749,91.679437,36542,753493.0
3,2304,96.046760,324.674608,1988.298706,4.828597,34.868880,22835.113037,803.744535,280.317435,174.098002,...,19.388554,207.075254,398.847263,297.243056,264.444721,402.037646,480.882159,141.363076,0,788388.0
4,2305,409.620345,305.099076,4802.696167,3.447335,36.600089,24900.542114,181.728030,302.311367,74.844749,...,74.631317,144.220043,98.590793,235.221391,71.364635,123.679642,689.004072,115.795126,0,589542.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,503.544596,146.570105,0.000000,1.715694,8.873014,21781.166748,353.886426,195.362847,11.992129,...,11.992129,11.992129,142.753944,216.870280,11.992129,345.732668,750.976438,76.298663,16191,585160.0
1872,1457,467.353819,181.912726,0.000000,1.477592,9.128512,29556.708984,515.272934,110.147483,41.028907,...,61.476908,61.497648,81.975277,646.026714,221.583073,15.504156,1155.758488,78.005631,23189,585160.0
1873,1458,39.430887,370.992033,29576.531372,1.241584,16.153486,1807.035522,533.787400,408.563075,29.649693,...,93.666293,29.649693,87.815255,615.073449,159.966721,29.649693,333.873145,94.340591,27963,635260.0
1874,1459,251.746998,182.598290,0.000000,1.241584,16.153486,21792.173340,449.461202,206.992487,85.553443,...,83.299103,68.921890,222.186216,1145.074644,393.971232,444.854015,1223.915921,33.956111,14512,605464.0


## 인물, 건물 데이터 병합
* 건축면적 : 대여소 반경 300m 내 합계 
* 건폐육 : 대여소 반경 300m 내 합계
* 공시지가 : 대여소 반경 300m 내 합계
* 용적율 : 대여소 반경 300m 내 합계
* 높이 : 대여소 반경 300m 내 합계
* 연면적 : 대여소 반경 300m 내 합계
* 지상층수 : 대여소 반경 300m 내 합계
* 대지면적 : 대여소 반경 300m 내 합계
* 거주인구 : 대여소 반경 300m 내 합계

In [95]:
people=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/인구데이터/최종대여소_건축물_인구_50.csv')

In [96]:
people.columns=['건축면적', '건폐율', '공시지가', '지하층수','용적율',
       '높이', '연면적', '지상층수', '대지면적', '인구', 'geometry', '대여소번호']

In [97]:
people=people[['대여소번호','건축면적', '건폐율', '공시지가', '지하층수','용적율',
       '높이', '연면적', '지상층수', '대지면적', '인구']]

In [107]:
people.head()

,대여소번호,건축면적,건폐율,공시지가,지하층수,용적율,높이,연면적,지상층수,대지면적,인구
0,1128,7297.84,488.85,47707848.04,11.27,2123.34,156.46,10705.97,44.34,3803.69,54.187500
1,1148,1264.66,487.11,25781752.92,9.27,1517.63,107.10,4842.88,32.16,2665.56,69.583333
2,1137,3098.96,354.60,26203467.58,10.03,1388.00,185.39,25935.65,56.26,21022.32,191.750000
3,835,1556.25,519.66,93329026.29,12.18,2171.98,162.05,11066.35,30.08,4434.67,12.214286
4,827,31.40,0.00,20615000.00,0.00,0.00,0.00,31.40,1.00,0.00,2.592697


In [99]:
rental=rental.merge(people,on='대여소번호',how='left')

In [108]:
rental.head()

,대여소번호,subway,park,상업지역,weighted_LEN_mean,mean_width,주거,school,sports,adult,...,공시지가,지하층수,용적율,높이,연면적,지상층수,대지면적,인구,starbucks,PB
0,2301,711.946112,252.411658,0.000000,0.845528,25.347815,14677.222656,87.435847,153.147202,106.104049,...,1.245851e+08,7.32,536.14,61.94,49065.67,45.83,1445.89,0.000000,4.0,1.0
1,2302,260.995145,317.618601,4691.636963,2.966590,40.044479,26691.501099,752.191921,268.035145,27.018746,...,1.789425e+08,13.48,2457.91,212.74,17192.30,50.36,4236.98,19.200000,7.0,2.0
2,2303,146.357936,145.560776,3866.095459,3.572432,39.651156,27516.596802,831.058272,179.164630,73.200264,...,1.625719e+08,16.70,2333.21,211.45,26948.84,55.05,5301.25,14.465517,5.0,2.0
3,2304,96.046760,324.674608,1988.298706,4.828597,34.868880,22835.113037,803.744535,280.317435,174.098002,...,9.064238e+07,16.13,2393.47,211.08,35039.55,54.41,7360.67,15.419355,5.0,5.0
4,2305,409.620345,305.099076,4802.696167,3.447335,36.600089,24900.542114,181.728030,302.311367,74.844749,...,1.672497e+08,14.75,2665.58,253.51,27188.62,57.53,6219.83,11.108108,4.0,2.0


## 프렌차이즈 독립변수 생성
* 스타벅스 : 반경 300m 내의 count
* 파리바게트 : 반경 300m 내의 count

In [101]:
star=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/대여소_스타벅스_파리바게트_250.csv')

In [102]:
star=star[['ID','starbucks','PB']]

In [103]:
star.columns=['대여소번호','starbucks','PB']

In [104]:
star.fillna(0,inplace=True)
rental=rental.merge(star,on='대여소번호')

In [109]:
rental.head()

,대여소번호,subway,park,상업지역,weighted_LEN_mean,mean_width,주거,school,sports,adult,...,공시지가,지하층수,용적율,높이,연면적,지상층수,대지면적,인구,starbucks,PB
0,2301,711.946112,252.411658,0.000000,0.845528,25.347815,14677.222656,87.435847,153.147202,106.104049,...,1.245851e+08,7.32,536.14,61.94,49065.67,45.83,1445.89,0.000000,4.0,1.0
1,2302,260.995145,317.618601,4691.636963,2.966590,40.044479,26691.501099,752.191921,268.035145,27.018746,...,1.789425e+08,13.48,2457.91,212.74,17192.30,50.36,4236.98,19.200000,7.0,2.0
2,2303,146.357936,145.560776,3866.095459,3.572432,39.651156,27516.596802,831.058272,179.164630,73.200264,...,1.625719e+08,16.70,2333.21,211.45,26948.84,55.05,5301.25,14.465517,5.0,2.0
3,2304,96.046760,324.674608,1988.298706,4.828597,34.868880,22835.113037,803.744535,280.317435,174.098002,...,9.064238e+07,16.13,2393.47,211.08,35039.55,54.41,7360.67,15.419355,5.0,5.0
4,2305,409.620345,305.099076,4802.696167,3.447335,36.600089,24900.542114,181.728030,302.311367,74.844749,...,1.672497e+08,14.75,2665.58,253.51,27188.62,57.53,6219.83,11.108108,4.0,2.0


In [110]:
rental.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/최종 독립변수(변환전).csv',encoding='cp949',index=False)